In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('spark://cm1:7077').appName('tutorial').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 22:15:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
arquivo = spark.read.option('header', True) \
                    .option('inferSchema' , True) \
                    .csv('hdfs://chococino:9000/user/hadoop/brazil-case')
arquivo

DataFrame[epi_week: int, date: timestamp, state: string, city: string, ibgeID: int, newDeaths: int, deaths: int, newCases: int, totalCases: int, deaths_per_100k_inhabitants: double, totalCases_per_100k_inhabitants: double, deaths_by_totalCases: double]

In [9]:
arquivo.head()

Row(epi_week='152', date='2022-01-01', state='GO', city='Abadia de Goiás/GO', ibgeID='5200050', newDeaths='0', deaths='44', newCases='1', totalCases='2003', deaths_per_100k_inhabitants='480.454', totalCases_per_100k_inhabitants='21871.588', deaths_by_totalCases='0.022')

In [11]:
arquivo.count()

2042905

In [12]:
arquivo.filter(arquivo.state == 'DF').take(10)

[Row(epi_week='152', date='2022-01-01', state='DF', city='Brasília/DF', ibgeID='5300108', newDeaths='0', deaths='11108', newCases='0', totalCases='519811', deaths_per_100k_inhabitants='358.980', totalCases_per_100k_inhabitants='16798.850', deaths_by_totalCases='0.021'),
 Row(epi_week='201', date='2022-01-02', state='DF', city='Brasília/DF', ibgeID='5300108', newDeaths='0', deaths='11108', newCases='0', totalCases='519811', deaths_per_100k_inhabitants='358.980', totalCases_per_100k_inhabitants='16798.850', deaths_by_totalCases='0.021'),
 Row(epi_week='201', date='2022-01-03', state='DF', city='Brasília/DF', ibgeID='5300108', newDeaths='2', deaths='11110', newCases='727', totalCases='520538', deaths_per_100k_inhabitants='359.044', totalCases_per_100k_inhabitants='16822.344', deaths_by_totalCases='0.021'),
 Row(epi_week='201', date='2022-01-04', state='DF', city='Brasília/DF', ibgeID='5300108', newDeaths='2', deaths='11112', newCases='602', totalCases='521140', deaths_per_100k_inhabitants

In [18]:
arquivo.select(arquivo.state, arquivo.newCases) \
       .groupBy(arquivo.state) \
       .sum('newCases').show()

+-----+-------------+
|state|sum(newCases)|
+-----+-------------+
|   SC|       712618|
|   RO|       187047|
|   PI|        79063|
|   AM|       191051|
|   RR|        52159|
|   GO|       876082|
|   TO|       124009|
|   MT|       304817|
|   SP|      1859225|
|   ES|       681272|
|   PB|       235792|
|   RS|      1388454|
|   MS|       212694|
|   AL|        93096|
|TOTAL|     14042646|
|   MG|      1855437|
|   PA|       235523|
|   BA|       498205|
|   SE|        79751|
|   PE|       477254|
+-----+-------------+
only showing top 20 rows



In [4]:
from pyspark.sql.functions import sum

In [7]:
arquivo.select(arquivo.state, arquivo.newDeaths, arquivo.newCases) \
       .filter(arquivo.newCases > 0) \
       .groupBy(arquivo.state) \
       .agg((sum(arquivo.newDeaths) / sum(arquivo.newCases)).alias('Mortes por Casos')) \
       .show()

+-----+--------------------+
|state|    Mortes por Casos|
+-----+--------------------+
|   SC|0.003079077773441885|
|   RO|0.002937666391260...|
|   PI|0.005055480378890392|
|   AM|0.002889258907683769|
|   RR|0.001614902779590...|
|   GO|0.003293138375180...|
|   TO|0.008819925132358714|
|   MT|0.003875135821154...|
|   SP|0.011299429213640414|
|   PB|0.003051345192028...|
|   ES|0.002250507618785...|
|   RS|0.003393296755461...|
|   MS|0.004393935176532807|
|   AL|0.006543064450669...|
|TOTAL|0.005324779959560328|
|   MG|0.002377899676836...|
|   PA|0.006839803817290079|
|   BA|0.003251790961011...|
|   SE|0.003662380660815...|
|   PE|-0.00472075023156...|
+-----+--------------------+
only showing top 20 rows



In [9]:
spark.stop()